In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/data/data4.zip", 'r')
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/colab_notebooks/data/10_comp_png.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
!ls /tmp/10_comp_png

hr  lr


In [ ]:
!ls /tmp/20_comp_png/hr

10000.png  13374.png  16747.png  20118.png  23491.png  26864.png  30235.png  33608.png	6634.png
10001.png  13375.png  16748.png  20119.png  23492.png  26865.png  30236.png  33609.png	6635.png
10002.png  13376.png  16749.png  2011.png   23493.png  26866.png  30237.png  3360.png	6636.png
10003.png  13377.png  1674.png	 20120.png  23494.png  26867.png  30238.png  33610.png	6637.png
10004.png  13378.png  16750.png  20121.png  23495.png  26868.png  30239.png  33611.png	6638.png
10005.png  13379.png  16751.png  20122.png  23496.png  26869.png  3023.png   33612.png	6639.png
10006.png  1337.png   16752.png  20123.png  23497.png  2686.png   30240.png  33613.png	663.png
10007.png  13380.png  16753.png  20124.png  23498.png  26870.png  30241.png  33614.png	6640.png
10008.png  13381.png  16754.png  20125.png  23499.png  26871.png  30242.png  33615.png	6641.png
10009.png  13382.png  16755.png  20126.png  2349.png   26872.png  30243.png  33616.png	6642.png
1000.png   13383.png  16756.png  20127.png 

In [ ]:
!rm -r /content/drive/MyDrive/colab_notebooks/data/results_20_comp_png

In [ ]:
!mkdir /content/drive/MyDrive/colab_notebooks/data/results_10_comp_png

In [ ]:
!ls

drive  results	sample_data


In [ ]:
import glob
from PIL import Image
import matplotlib.pyplot as plt
import random
import numpy as np
from tensorflow.keras import Input
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, Add, Dense
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.17.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.17.0


In [ ]:
!pip install tensorflow==2.15.0


y
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installat

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
'''
def residual_block(x):
    """
    Residual block
    """
    filters = [64, 64]
    kernel_size = 3
    strides = 1
    padding = "same"
    momentum = 0.8
    activation = "relu"

    res = Conv2D(filters=filters[0], kernel_size=kernel_size,
                 strides=strides, padding=padding)(x)
    res = Activation(activation=activation)(res)
    res = BatchNormalization(momentum=momentum)(res)

    res = Conv2D(filters=filters[1], kernel_size=kernel_size,
                 strides=strides, padding=padding)(res)
    res = BatchNormalization(momentum=momentum)(res)

    # Add res and x
    res = Add()([res, x])
    return res
'''

from tensorflow.keras.layers import Multiply

from tensorflow.keras.layers import Concatenate , Lambda # Import the Concatenate layer from Keras

def dense_block(input):

  x1 = Conv2D(64, kernel_size=3, strides=1, padding='same')(input)
  x1 = LeakyReLU(0.2)(x1)
  x1 = Concatenate()([input, x1])

  x2 = Conv2D(64, kernel_size=3, strides=1, padding='same')(x1)
  x2 = LeakyReLU(0.2)(x2)
  x2 = Concatenate()([input, x1, x2])

  x3 = Conv2D(64, kernel_size=3, strides=1, padding='same')(x2)
  x3 = LeakyReLU(0.2)(x3)
  x3 = Concatenate()([input, x1, x2, x3])

  x4 = Conv2D(64, kernel_size=3, strides=1, padding='same')(x3)
  x4 = LeakyReLU(0.2)(x4)
  x4 = Concatenate()([input, x1, x2, x3, x4])

  x5 = Conv2D(64, kernel_size=3, strides=1, padding='same')(x4)
  x5 = Lambda(lambda x: x * 0.2)(x5)
  x = Add()([x5, input])
  return x

#----------------
'''
  x5 = Conv2D(64, kernel_size=3, strides=1, padding='same')(x4)
  x5 = LeakyReLU(0.2)(x5)
  x5 = Concatenate()([input, x1, x2, x3, x4,x5])

  x6 = Conv2D(64, kernel_size=3, strides=1, padding='same')(x5)
  x6 = LeakyReLU(0.2)(x6)
  x6 = Concatenate()([input, x1, x2, x3, x4,x5,x6])

  x7 = Conv2D(64, kernel_size=3, strides=1, padding='same')(x6)
  x7 = LeakyReLU(0.2)(x7)
  x7 = Concatenate()([input, x1, x2, x3, x4,x5,x6,x7])


  x8 = Conv2D(64, kernel_size=3, strides=1, padding='same')(x7)
  x8 = Lambda(lambda x: x * 0.2)(x8)
  x = Add()([x8, input])
  '''
#----------------------------------------------------


def residual_block(input):

    x = dense_block(input)
    x = dense_block(x)
    x = dense_block(x)
    x = Lambda(lambda x: x * 0.2)(x)
    out = Add()([x, input])
    return out

def channel_attention(input, reduction_ratio=16):
    # Global Average Pooling
    avg_pool = tf.reduce_mean(input, axis=[1, 2], keepdims=True)
    # Dense layers
    dense1 = Dense(units=input.shape[-1] // reduction_ratio, activation='relu')(avg_pool)
    dense2 = Dense(units=input.shape[-1], activation='sigmoid')(dense1)
    # Scale
    return Multiply()([input, dense2])

def residual_channel_attention_block(input):
    x = Conv2D(64, kernel_size=3, strides=1, padding='same')(input)
    x = Activation('relu')(x)
    x = Conv2D(64, kernel_size=3, strides=1, padding='same')(x)
    x = channel_attention(x)
    return Add()([x, input])

def residual_group(input, num_blocks):
    x = input
    for _ in range(num_blocks):
        x = residual_channel_attention_block(x)
    return Add()([x, input])

def build_rcan(num_residual_groups=10, num_residual_blocks=20):
    input_layer = Input(shape=(None, None, 3))

    x = Conv2D(64, kernel_size=3, strides=1, padding='same')(input_layer)

    for _ in range(num_residual_groups):
        x = residual_group(x, num_residual_blocks)

    x = Conv2D(3, kernel_size=3, strides=1, padding='same')(x)
    x = Add()([x, input_layer])

    # Upsampling
    x = UpSampling2D(size=2)(x)
    x = Conv2D(256, kernel_size=3, strides=1, padding='same')(x)
    x = Activation('relu')(x)
    x = UpSampling2D(size=2)(x)
    x = Conv2D(256, kernel_size=3, strides=1, padding='same')(x)
    x = Activation('relu')(x)
    output = Conv2D(3, kernel_size=3, strides=1, padding='same', activation='tanh')(x)
    model = Model(inputs=input_layer, outputs=output)
    return model

def build_generator():
    """
    Create a generator network using the hyperparameter values defined below
    :return:
    """
    residual_blocks = 16
    momentum = 0.8
    input_shape = (None, None, 3)

    # Input Layer of the generator network
    input_layer = Input(shape=input_shape)

    # Add the pre-residual block
    gen1 = Conv2D(filters=64, kernel_size=9, strides=1, padding='same',
                  activation='relu')(input_layer)

    # Add 16 residual blocks
    res = residual_block(gen1)
    for i in range(residual_blocks - 1):
        res = residual_block(res)

    # Add the post-residual block
    gen2 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(res)
    gen2 = BatchNormalization(momentum=momentum)(gen2)

    # Take the sum of the output from the pre-residual block(gen1) and  the post-residual block(gen2)
    gen3 = Add()([gen2, gen1])

    # Add an upsampling block
    gen4 = UpSampling2D(size=2)(gen3)
    gen4 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same')(gen4)
    gen4 = Activation('relu')(gen4)

    # Add another upsampling block
    gen5 = UpSampling2D(size=2)(gen4)
    gen5 = Conv2D(filters=256, kernel_size=3, strides=1,
                  padding='same')(gen5)
    gen5 = Activation('relu')(gen5)

    # Output convolution layer
    gen6 = Conv2D(filters=3, kernel_size=9, strides=1, padding='same')(gen5)
    output = Activation('tanh')(gen6)

    # Keras model
    model = Model(inputs=[input_layer], outputs=[output],
                  name='generator')
    return model


def build_discriminator():
    """
    Create a discriminator network using the hyperparameter values defined below
    :return:
    """
    leakyrelu_alpha = 0.2
    momentum = 0.8
    input_shape = (256, 256, 3)

    input_layer = Input(shape=input_shape)

    # Add the first convolution block
    dis1 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(input_layer)
    dis1 = LeakyReLU(alpha=leakyrelu_alpha)(dis1)

    # Add the 2nd convolution block
    dis2 = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(dis1)
    dis2 = LeakyReLU(alpha=leakyrelu_alpha)(dis2)
    dis2 = BatchNormalization(momentum=momentum)(dis2)

    # Add the third convolution block
    dis3 = Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(dis2)
    dis3 = LeakyReLU(alpha=leakyrelu_alpha)(dis3)
    dis3 = BatchNormalization(momentum=momentum)(dis3)

    # Add the fourth convolution block
    dis4 = Conv2D(filters=128, kernel_size=3, strides=2, padding='same')(dis3)
    dis4 = LeakyReLU(alpha=leakyrelu_alpha)(dis4)
    dis4 = BatchNormalization(momentum=0.8)(dis4)

    # Add the fifth convolution block
    dis5 = Conv2D(256, kernel_size=3, strides=1, padding='same')(dis4)
    dis5 = LeakyReLU(alpha=leakyrelu_alpha)(dis5)
    dis5 = BatchNormalization(momentum=momentum)(dis5)

    # Add the sixth convolution block
    dis6 = Conv2D(filters=256, kernel_size=3, strides=2, padding='same')(dis5)
    dis6 = LeakyReLU(alpha=leakyrelu_alpha)(dis6)
    dis6 = BatchNormalization(momentum=momentum)(dis6)

    # Add the seventh convolution block
    dis7 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(dis6)
    dis7 = LeakyReLU(alpha=leakyrelu_alpha)(dis7)
    dis7 = BatchNormalization(momentum=momentum)(dis7)

    # Add the eight convolution block
    dis8 = Conv2D(filters=512, kernel_size=3, strides=2, padding='same')(dis7)
    dis8 = LeakyReLU(alpha=leakyrelu_alpha)(dis8)
    dis8 = BatchNormalization(momentum=momentum)(dis8)

    # Add a dense layer
    dis9 = Dense(units=1024)(dis8)
    dis9 = LeakyReLU(alpha=0.2)(dis9)

    # Last dense layer - for classification
    output = Dense(units=1, activation='sigmoid')(dis9)

    model = Model(inputs=[input_layer], outputs=[output], name='discriminator')
    return model






def build_vgg():
    """
    Build the VGG network to extract image features
    """
    input_shape = (256, 256, 3)
    # Load a pre-trained VGG19 model trained on 'Imagenet' dataset
    vgg = VGG19(weights='imagenet',include_top=False,input_shape=input_shape)
    for i, layer in enumerate(vgg.layers):
      print(i, layer.name)
      #if i == 20:
        #vgg.outputs = [layer.output]
    vgg.trainable = False

    # Set the output to be the 'block5_conv4' layer features
    #vgg.outputs = [vgg.layers[20].output]
    #output_layer = vgg.get_layer('block5_conv4').output
    output_layer = vgg.get_layer('block3_conv3').output
    print(vgg.summary())
    # Use the vgg.input as the input layer for the model
    input_layer = vgg.input
    #input_layer = Input(shape=input_shape)
    # Extract features
    #features = vgg(input_layer)
    # Create a Keras model
    #model = Model(inputs=[input_layer], outputs=[features])
    model = Model(input_layer, output_layer)
    return model


'''
def build_vgg():
    """
    Build the VGG network to extract image features
    """
    input_shape = (256, 256, 3)
    vgg = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

    # Set the layers to be frozen
    for layer in vgg.layers[:8]:
        layer.trainable = False

    # Set the layers to be trainable
    for layer in vgg.layers[8:]:
        layer.trainable = True

    output_layer = vgg.get_layer('block3_conv3').output
    input_layer = vgg.input
    print(vgg.summary())
    model = Model(input_layer, output_layer)
    return model

'''
'''
from keras.applications import ResNet152

def build_resnet152():
    """
    Build the ResNet152 network to extract image features
    """
    input_shape = (256, 256, 3)

    # Load the pre-trained ResNet152 model trained on 'Imagenet' dataset
    resnet152 = ResNet152(weights='imagenet', include_top=False, input_shape=input_shape)

    # Make the model non-trainable
    resnet152.trainable = False

    # Choose the layer for feature extraction
    output_layer = resnet152.get_layer('conv5_block3_out').output

    model = Model(resnet152.input, output_layer)

    return model
'''
def build_adversarial_model(generator, discriminator, vgg):
    input_low_resolution = Input(shape=(None, None, 3))

    fake_hr_images = generator(input_low_resolution)
    fake_features = vgg(fake_hr_images)

    discriminator.trainable = False

    output = discriminator(fake_hr_images)

    model = Model(inputs=[input_low_resolution],
                  outputs=[output, fake_features])

    for layer in model.layers:
        print(layer.name, layer.trainable)

    print(model.summary())
    return model

In [ ]:
def load_images(filename,new_size):
    image = Image.open(filename)
    resized_image = image.resize(new_size[:-1],Image.BICUBIC)
    resized_image=np.array(resized_image)
    # Check if the image is grayscale (2D array)
    if len(resized_image.shape) == 2:
        # Add a third channel by repeating the grayscale data
        resized_image = np.stack((resized_image,) * 3, axis=-1)
    return resized_image

def sample_images(data_dir, batch_size, high_resolution_shape, low_resolution_shape):
    # Make a list of all images inside the data directory
    all_hr_images = sorted(glob.glob(data_dir + "hr/*.png"))
    all_lr_images = sorted(glob.glob(data_dir + "lr/*.png"))
    # Ensure we have paired the images correctly
    assert len(all_hr_images) == len(all_lr_images)

    # Pair the corresponding hr and lr images
    paired_images = list(zip(all_hr_images, all_lr_images))
    # Shuffle the paired images
    random.shuffle(paired_images)


    # Split the images into batches
    images_batch = paired_images[0: batch_size]

    low_resolution_images = []
    high_resolution_images = []

    for img in images_batch:
        # Get an ndarray of the current images
        hr_batch_files, lr_batch_files = img
        hr_batch = load_images(hr_batch_files,high_resolution_shape)
        lr_batch = load_images(lr_batch_files,low_resolution_shape)

        # Do a random horizontal flip
        if np.random.random() < 0.5:
            hr_batch = np.fliplr(hr_batch)
            lr_batch = np.fliplr(lr_batch)
        print("hr_batch",hr_batch.shape)
        print("lr_batch",lr_batch.shape)
        high_resolution_images.append(hr_batch)
        low_resolution_images.append(lr_batch)

    # Convert the lists to Numpy NDArrays
    return np.array(high_resolution_images), np.array(low_resolution_images)




def save_images(low_resolution_image, original_image, generated_image, path):
    """
    Save low-resolution, high-resolution(original) and
    generated high-resolution images in a single image
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 3, 1)
    ax.imshow(low_resolution_image.astype('uint8'))
    ax.axis("off")
    ax.set_title("Low-resolution")

    ax = fig.add_subplot(1, 3, 2)
    ax.imshow(original_image.astype('uint8'))
    ax.axis("off")
    ax.set_title("Original")

    ax = fig.add_subplot(1, 3, 3)
    ax.imshow(generated_image.astype('uint8'))
    ax.axis("off")
    ax.set_title("Generated")

    plt.savefig(path)

In [ ]:
import csv
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import sys
# Set the recursion limit to a higher value
sys.setrecursionlimit(10000)

results = []
d_loss_values = []
g_loss_values = []


print("entered main")
# Define hyperparameters
#data_dir = "/tmp/data4/"
data_dir = "/tmp/10_comp_png/"
epochs = 6000   ###1500
batch_size = 2
mode = 'train'

# Shape of low-resolution and high-resolution images
low_resolution_shape = (64, 64, 3)
high_resolution_shape = (256, 256, 3)
# Common optimizer for all networks
common_optimizer = Adam(0.0001, 0.5)   ###0.0002

if mode == 'train':
    print("entered training mode")
    vgg = build_vgg()
    vgg.trainable = False
    vgg.compile(loss='mse', optimizer=common_optimizer, metrics=
                ['accuracy'])

    discriminator = build_discriminator()

    # build the generator network
    generator =  build_rcan()  #build_generator()
    input_high_resolution = Input(shape=high_resolution_shape)
    input_low_resolution = Input(shape=low_resolution_shape)
    # Load models
    #generator.load_weights("generator.h5")
    #discriminator.load_weights("discriminator.h5")
    generated_high_resolution_images = generator(input_low_resolution)
    features = vgg(generated_high_resolution_images)
    # Extract feature maps of the generated images
    features = vgg(generated_high_resolution_images)

    # Make the discriminator network as non-trainable
    discriminator.trainable = False
    discriminator.compile(loss='mse', optimizer=common_optimizer,
                          metrics=['accuracy'])
    # Get the probability of generated high-resolution images
    probs = discriminator(generated_high_resolution_images)

    # Create and compile an adversarial model
    adversarial_model = Model([input_low_resolution, input_high_resolution], [probs, features])
    adversarial_model.compile(loss=['binary_crossentropy', 'mse'], loss_weights=[1e-3, 1], optimizer=common_optimizer)

    print("models built successfully")
    for epoch in range(epochs):
        print("Epoch:{}".format(epoch))

        """
        Train the discriminator network
        """

        # Sample a batch of images
        high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=batch_size,
                                                                      low_resolution_shape=low_resolution_shape,
                                                                      high_resolution_shape=high_resolution_shape)
        # Normalize images
        high_resolution_images = high_resolution_images / 127.5 - 1.
        low_resolution_images = low_resolution_images / 127.5 - 1.


       # print(high_resolution_images.shape)
       # print(low_resolution_images.shape)

        # Generate high-resolution images from low-resolution images
        generated_high_resolution_images = generator.predict(low_resolution_images)

        # Generate batch of real and fake labels
        real_labels = np.ones((batch_size, 16, 16, 1))
        fake_labels = np.zeros((batch_size, 16, 16, 1))

        # Train the discriminator network on real and fake images
        d_loss_real = discriminator.train_on_batch(high_resolution_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(generated_high_resolution_images, fake_labels)

        # Calculate total discriminator loss
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        d_loss_values.append(d_loss)
        print("d_loss:", d_loss)


        """
        Train the generator network
        """

        # Sample a batch of images
        high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=batch_size,
                                                                      low_resolution_shape=low_resolution_shape,
                                                                      high_resolution_shape=high_resolution_shape)
        # Normalize images
        high_resolution_images = high_resolution_images / 127.5 - 1.
        low_resolution_images = low_resolution_images / 127.5 - 1.

        # Extract feature maps for real high-resolution images
        image_features = vgg.predict(high_resolution_images)

        # Train the generator network
        g_loss = adversarial_model.train_on_batch([low_resolution_images, high_resolution_images],
                                         [real_labels, image_features])

        g_loss_values.append(g_loss)
        print("g_loss:", g_loss)

        # Sample and save images after every 100 epochs
        if epoch % 5 == 0:
            high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=batch_size,
                                                                          low_resolution_shape=low_resolution_shape,
                                                                           high_resolution_shape=high_resolution_shape)
            # Normalize images
            high_resolution_images_norm = high_resolution_images / 127.5 - 1.
            low_resolution_images_norm = low_resolution_images / 127.5 - 1.

            generated_images = generator.predict_on_batch(low_resolution_images_norm)

            for index, img in enumerate(generated_images):
                original_sr_image = np.round((img + 1) * 127.5).astype('uint8')

                #-------------- SSIM and PSNR --------------
                # Calculate PSNR and SSIM
                psnr_org_vs_gen = psnr(high_resolution_images[index], original_sr_image)
                ssim_org_vs_gen = ssim(high_resolution_images[index], original_sr_image, multichannel=True, win_size=3)

                 #-------------- SSIM and PSNR --------------


                # Resize the low_resolution_image to match the dimensions of high_resolution_images
                resized_lr_image = cv2.resize(low_resolution_images[index], (high_resolution_images[index].shape[1], high_resolution_images[index].shape[0]))


                psnr_org_vs_lr = psnr(high_resolution_images[index], resized_lr_image)
                ssim_org_vs_lr = ssim(high_resolution_images[index], resized_lr_image, multichannel=True, win_size=3)

                # Append results to the list
                results.append(["img_{}_{}".format(epoch, index), ssim_org_vs_gen, psnr_org_vs_gen, ssim_org_vs_lr, psnr_org_vs_lr])

                #----------------------------------------------
                save_images(low_resolution_images[index], high_resolution_images[index], original_sr_image,
                            path="/content/drive/MyDrive/colab_notebooks/data/results_10_comp_png/img_{}_{}".format(epoch, index))



    # Save models
    generator.save_weights("/content/drive/MyDrive/colab_notebooks/data/generator_10_comp_png.h5")
    discriminator.save_weights("/content/drive/MyDrive/colab_notebooks/data/discriminator_10_comp_png.h5")




entered main
entered training mode
0 input_6
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_conv4
11 block3_pool
12 block4_conv1
13 block4_conv2
14 block4_conv3
15 block4_conv4
16 block4_pool
17 block5_conv1
18 block5_conv2
19 block5_conv3
20 block5_conv4
21 block5_pool
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                        

<ipython-input-6-1332d5816cc4>:56: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 18ms/step
g_loss: [59.46146011352539, 0.6912875175476074, 59.46076965332031]
hr_batch (256, 256, 3)
lr_batch (64, 64, 3)
hr_batch (256, 256, 3)
lr_batch (64, 64, 3)
Epoch:3836
hr_batch (256, 256, 3)
lr_batch (64, 64, 3)
hr_batch (256, 256, 3)
lr_batch (64, 64, 3)
1/1 [==============================] - 0s 201ms/step
d_loss: [0.25033945 0.43554688]
hr_batch (256, 256, 3)
lr_batch (64, 64, 3)
hr_batch (256, 256, 3)
lr_batch (64, 64, 3)
1/1 [==============================] - 0s 24ms/step
g_loss: [38.47330856323242, 0.6912765502929688, 38.472618103027344]
Epoch:3837
hr_batch (256, 256, 3)
lr_batch (64, 64, 3)
hr_batch (256, 256, 3)
lr_batch (64, 64, 3)
1/1 [==============================] - 0s 212ms/step
d_loss: [0.25035553 0.36230469]
hr_batch (256, 256, 3)
lr_batch (64, 64, 3)
hr_batch (256, 256, 3)
lr_batch (64, 64, 3)
1/1 [==============================] - 0s 18ms/step
g_loss: [63.32870864868164